In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

class Fundamentus:

    def __init__(self,Ftype:int,Ativo:str):
        """
        :param Ftype: Tipo de Busca (1: informações gerais, 2:Dividendos)
        :param Ativo: Nome do ativo (Colocar tudo em maiusculo, ex: 'PETR4')
        """
        self.Ftype=Ftype
        self.Ativo=Ativo

    def Get_information(self):
        """
            Faz um request no fundamentus e retorna as informações do ativo.

        :return: Retorna um dataset com o tipo de busca (Dividendos ou Informações Gerais)
        """
        F21=0
        "Buscar info gerais"
        if self.Ftype==1:

            try:
                headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 OPR/82.0.4227.50'}
                source = requests.get(f"https://fundamentus.com.br/detalhes.php?papel={self.Ativo}",headers=headers).content
                
                soup = BeautifulSoup(source, 'html.parser')
                contents=soup.find_all(attrs={'class':'label'})
                contents2=soup.find_all(attrs={'class':'data'})
                Fund=pd.DataFrame()
                
                for x in range(len(contents)):
                    label=contents[x].text
                    label=label.replace("?",'')
                    data=contents2[x].text
                    data=data.replace("\n-", "")
                    data = data.replace("\n", "")
                    data_row=pd.DataFrame({"Info":label,"Data":data},index=[0])
                    Fund=Fund.append(data_row)

                Fund=Fund.reset_index(drop=True)
                Fund["Duplicates"]=Fund["Info"].duplicated()
                F=Fund[Fund["Duplicates"]==True]
                F21=Fund
                FEnd=pd.DataFrame()
                for x in F["Info"]:
                    F21 = F21[(F21["Info"] !=x)]
                    F2 = Fund[(Fund["Duplicates"] == False)&(Fund["Info"]==x)]
                    F23 = Fund[(Fund["Duplicates"] == True) & (Fund["Info"] == x)]

                    F2["Info"]=f"Relatorio 12 meses {x}"
                    F23["Info"] = f"Relatorio 3 meses {x}"

                    FEnd=pd.concat([F2,F23,FEnd])

                F21=F21.append(FEnd).reset_index(drop=True)
                F21=F21.drop(columns=["Duplicates"])
                F21=F21[F21["Info"]!=""].reset_index(drop=True)
            except:
                print("Não foi possível buscar dados do Fundamentus!")

        "Buscar dividendos"
        if self.Ftype == 2:
            headers = {
                'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 OPR/82.0.4227.50'
            }
            source = requests.get(f"https://fundamentus.com.br/proventos.php?papel={self.Ativo}&tipo=2",headers=headers).content
            soup = BeautifulSoup(source, 'html.parser')
           
            teste = soup.find_all("h1")[0].text
   
            prove="Go"
            if teste=="Nenhum provento encontrado ":
                prove="no"

                source = requests.get(f"https://fundamentus.com.br/fii_proventos.php?papel={self.Ativo}&tipo=2",headers=headers).content
                soup = BeautifulSoup(source, 'html.parser')
                teste = soup.find_all("h1")[0].text
                if teste == "Nenhum provento encontrado ":
                    prove = "No"
                else:
                    prove="Go"
                    if prove == "Go":
                        soup = soup.find('div', id='tabela-completa')
                        contents = soup.find_all("td")
                        # contents2 = soup.find_all(attrs={'class': 'data'})
                        Fund = pd.DataFrame()
                        g = 0
                        Divi = pd.DataFrame()

                        for x in contents:

                            g = g + 1
                            if g == 1:
                                data = x.text
                            if g == 2:
                                tipo = x.text
                            if g == 3:
                                data2 = x.text
                            if g == 4:
                                valor = x.text
                                g = 0
                                new_row = pd.DataFrame({"Data": data, "Valor pago": valor, "Tipo de dividendos": tipo,"Data de pagamento": data2}, index=[0])
                                Divi = Divi.append(new_row)
                                F21=Divi
            else:
                if prove=="Go":
                    soup = soup.find('div', id='tabela-completa')
                    contents = soup.find_all("td")

                    #contents2 = soup.find_all(attrs={'class': 'data'})
                    Fund = pd.DataFrame()
                    g=0
                    Divi=pd.DataFrame()
                    for x in contents:

                        g=g+1
                        if g==1:
                            data=x.text
                        if g==2:
                            valor = x.text
                        if g==3:
                            tipo=x.text
                        if g==4:
                            data2 = x.text
                        if g==5:
                            qntd = x.text
                            g=0
                            new_row=pd.DataFrame({"Data":data,"Valor pago":valor,"Tipo de dividendos":tipo,"Data de pagamento":data2,"Qauntidade de Ações":qntd},index=[0])
                            Divi=Divi.append(new_row)
                            F21 = Divi

        return F21

    def save(self,data):
        """
            Salva o Dataset.

        :param data: Dataset para salvar
        """
        self.data=data
        if self.Ftype==1:
            data.to_csv(f"Relatório de Bignumbers {self.Ativo}.csv", sep=";", index=False, encoding='latin-1', mode="w",decimal=',')
        if self.Ftype==2:
            data.to_csv(f"Relatório de Dividendos {self.Ativo}.csv", sep=";", index=False, encoding='latin-1', mode="w",decimal=',')







In [20]:
tick=pd.read_excel(fr"Depara\tick.xlsx")
Cuspidor_div=pd.DataFrame()
Cuspidor_inf=pd.DataFrame()

In [27]:
tickes=tick['Ticker'].drop_duplicates()



for x in tickes:
       
        try:
            D=Fundamentus(2,x)
            Div=D.Get_information()
            Div=Div.reset_index(drop=True)
            Div["tick"]=x
            Cuspidor_div=Cuspidor_div.append(Div, ignore_index=True)
        except:
            print("Sem dividendos")

for x in tickes:
       
        try:
            D1=Fundamentus(1,x)
            Inf=D1.Get_information()
            Inf=Inf.reset_index(drop=True)
            Inf["tick"]=x
            Cuspidor_inf=Cuspidor_inf.append(Inf, ignore_index=True)
        except:
            print("Sem fundamentus")        
        
        

Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem dividendos
Sem divide

In [12]:
print(Cuspidor_div)


             Data Valor pago Tipo de dividendos Data de pagamento  \
0      27/04/2020     0,0872          DIVIDENDO                -    
1      25/04/2019     0,0845          DIVIDENDO       07/05/2019    
2      30/06/2023     0,8360    JRS CAP PROPRIO       16/08/2023    
3      29/12/2022     0,3700    JRS CAP PROPRIO       13/01/2023    
4      30/09/2022     0,3620    JRS CAP PROPRIO       13/10/2022    
...           ...        ...                ...               ...   
16746  03/04/2012     0,2026          DIVIDENDO                -    
16747  19/04/2011     0,4674          DIVIDENDO                -    
16748  19/04/2010     0,3884          DIVIDENDO                -    
16749  28/04/2009     0,2273          DIVIDENDO                -    
16750  25/04/2008     0,0579          DIVIDENDO                -    

      Qauntidade de Ações   tick  
0                       1  AALR3  
1                       1  AALR3  
2                       1  ABCB4  
3                       1  ABCB

In [29]:
print(Cuspidor_inf[Cuspidor_inf['tick']=="PETR4"])
Cuspidor_inf.to_csv(fr"C:\Users\marqu\Desktop\Marcus\Estudos Finanças\IA_FINAN-AS_APS_2023\Cuspidão\cuspidor_inf.csv",decimal=',',sep=';', encoding='latin-1',index=False)
Cuspidor_div.to_csv(fr"C:\Users\marqu\Desktop\Marcus\Estudos Finanças\IA_FINAN-AS_APS_2023\Cuspidão\cuspidor_div.csv",decimal=',',sep=';', encoding='latin-1',index=False)

                                     Info                               Data  \
23747                               Papel                              PETR4   
23748                             Cotação                              29,10   
23749                                Tipo                                 PN   
23750                        Data últ cot                         19/07/2023   
23751                             Empresa                       PETROBRAS PN   
23752                          Min 52 sem                              17,46   
23753                               Setor    Petróleo, Gás e Biocombustíveis   
23754                          Max 52 sem                              31,85   
23755                            Subsetor  Exploração, Refino e Distribuição   
23756                      Vol $ méd (2m)                      1.622.090.000   
23757                    Valor de mercado                    379.595.000.000   
23758              Últ balanço processad

In [28]:
print(Cuspidor_inf[Cuspidor_inf['tick']=="AZUL4"])

                                    Info              Data   tick
4222                               Papel             AZUL4  AZUL4
4223                             Cotação             17,23  AZUL4
4224                                Tipo                PN  AZUL4
4225                        Data últ cot        19/07/2023  AZUL4
4226                             Empresa           AZUL PN  AZUL4
4227                          Min 52 sem              6,79  AZUL4
4228                               Setor        Transporte  AZUL4
4229                          Max 52 sem             21,86  AZUL4
4230                            Subsetor  Transporte Aéreo  AZUL4
4231                      Vol $ méd (2m)       266.691.000  AZUL4
4232                    Valor de mercado     5.996.210.000  AZUL4
4233              Últ balanço processado        31/03/2023  AZUL4
4234                      Valor da firma    28.674.600.000  AZUL4
4235                          Nro. Ações       348.010.000  AZUL4
4236      